In [ ]:
import pandas as pd
import numpy as np

короче в ячейке ниже надо каждый раз подгружать с компа файл с исходной таблицей
я так и не нашла, как нормально считывать файлы в коллабе, но на гите будет проще, думаю

In [ ]:
import io
from google.colab import files
uploaded = files.upload()
data = pd.read_csv(io.StringIO(uploaded['forbes2000 (1).csv'].decode('utf-8')))
data.head()

Saving forbes2000 (1).csv to forbes2000 (1).csv


rank                                      name        country      sales  \
0   1.0                        Berkshire Hathaway  United States  $276.09 B   
1   2.0                                      ICBC          China  $208.13 B   
2   3.0  Saudi Arabian Oil Company (Saudi Aramco)   Saudi Arabia  $400.38 B   
3   4.0                            JPMorgan Chase  United States  $124.54 B   
4   5.0                   China Construction Bank          China  $202.07 B   

      profit       assets           MV                Industry  Founded  \
0    $89.8 B    $958.78 B    $741.48 B  Diversified Financials   1939.0   
1   $54.03 B  $5,518.51 B    $214.43 B                     NaN   1984.0   
2  $105.36 B    $576.04 B  $2,292.08 B    Oil & Gas Operations   1933.0   
3   $42.12 B  $3,954.69 B    $374.45 B                 Banking   2000.0   
4   $46.89 B  $4,746.95 B    $181.32 B                     NaN   1954.0   

          Headquaters                       CEO Employees Share price  
0     Omaha, Nebraska     Warren Edward Buffett   372,000     $318.05  
1             Beijing                  Liao Lin   434,089        None  
2             Dhahran  Amin bin Hasan Al-Nasser    68,493        None  
3  New York, New York               Jamie Dimon   270,000     $128.99  
4             Beijing            Jinliang Zhang   347,319        None

Итак, у нас есть несколько колонок с числовыми данными, но в них присутствуют другие символы помимо цифр (валюта и B или M). Надо избавиться от этих символов, чтобы было удобно анализировать.

In [ ]:
data['sales'] = data['sales'].apply(lambda x: x[1:])
data['profit'] = data['profit'].apply(lambda x: x[1:])
data['assets'] = data['assets'].apply(lambda x: x[1:])
data['MV'] = data['MV'].apply(lambda x: x[1:])
data['Share price'] = data['Share price'].apply(lambda x: x[1:] if x[0]=='$' else None)#избавляемся от значка доллара

Далее создаем вспомогательные колонки, в которых содержится "1", если число было написано в миллионах долларов (М), и "1000", если число написано в миллиардах (B), т.к. мы хотим привести все к милионам

In [ ]:
data['sales MB'] = data['sales'].apply(lambda x: 1 if x[-1] == 'M' else 1000 if x[-1] == 'B' else None)
data['assets MB'] = data['assets'].apply(lambda x: 1 if x[-1] == 'M' else 1000 if x[-1] == 'B' else None)
data['profit MB'] = data['profit'].apply(lambda x: 1 if x[-1] == 'M' else 1000 if x[-1] == 'B' else None)
data['MV MB'] = data['MV'].apply(lambda x: 1 if x[-1] == 'M' else 1000 if x[-1] == 'B' else None)

Удаляем "М" и "В", переводим колонки в float

In [ ]:
data['sales'] = data['sales'].apply(lambda x: float(x[:-1]))
data['profit'] = data['profit'].apply(lambda x: float(x[:-1]))
data['assets'] = data['assets'].apply(lambda x: float(x.replace(',', '')[:-1]))
data['MV'] = data['MV'].apply(lambda x: float(x.replace(',', '')[:-1]))
data['Share price'] = data['Share price'].apply(lambda x: float(x.replace(',', '')[:-1]) if x != None else None)

Умножаем на вспомогательные колонки, затем удаляем вспомогательные колонки

In [ ]:
data['sales']=data['sales']*data['sales MB']
data['assets']=data['assets']*data['assets MB']
data['profit']=data['profit']*data['profit MB']
data['MV']=data['MV']*data['MV MB']
data.head() #теперь sales, profit, assets и market value указаны в миллионах долларов

rank                                      name        country     sales  \
0   1.0                        Berkshire Hathaway  United States  276090.0   
1   2.0                                      ICBC          China  208130.0   
2   3.0  Saudi Arabian Oil Company (Saudi Aramco)   Saudi Arabia  400380.0   
3   4.0                            JPMorgan Chase  United States  124540.0   
4   5.0                   China Construction Bank          China  202070.0   

     profit     assets         MV                Industry  Founded  \
0   89800.0   958780.0   741480.0  Diversified Financials   1939.0   
1   54030.0  5518510.0   214430.0                     NaN   1984.0   
2  105360.0   576040.0  2292080.0    Oil & Gas Operations   1933.0   
3   42120.0  3954690.0   374450.0                 Banking   2000.0   
4   46890.0  4746950.0   181320.0                     NaN   1954.0   

          Headquaters                       CEO Employees  Share price  \
0     Omaha, Nebraska     Warren Edward Buffett   372,000        318.0   
1             Beijing                  Liao Lin   434,089          NaN   
2             Dhahran  Amin bin Hasan Al-Nasser    68,493          NaN   
3  New York, New York               Jamie Dimon   270,000        128.9   
4             Beijing            Jinliang Zhang   347,319          NaN   

   sales MB  assets MB  profit MB  MV MB  
0      1000       1000       1000   1000  
1      1000       1000       1000   1000  
2      1000       1000       1000   1000  
3      1000       1000       1000   1000  
4      1000       1000       1000   1000

In [ ]:
data = data.drop(columns = ['sales MB','assets MB','profit MB','MV MB'])
data.head()

rank                                      name        country     sales  \
0   1.0                        Berkshire Hathaway  United States  276090.0   
1   2.0                                      ICBC          China  208130.0   
2   3.0  Saudi Arabian Oil Company (Saudi Aramco)   Saudi Arabia  400380.0   
3   4.0                            JPMorgan Chase  United States  124540.0   
4   5.0                   China Construction Bank          China  202070.0   

     profit     assets         MV                Industry  Founded  \
0   89800.0   958780.0   741480.0  Diversified Financials   1939.0   
1   54030.0  5518510.0   214430.0                     NaN   1984.0   
2  105360.0   576040.0  2292080.0    Oil & Gas Operations   1933.0   
3   42120.0  3954690.0   374450.0                 Banking   2000.0   
4   46890.0  4746950.0   181320.0                     NaN   1954.0   

          Headquaters                       CEO Employees  Share price  
0     Omaha, Nebraska     Warren Edward Buffett   372,000        318.0  
1             Beijing                  Liao Lin   434,089          NaN  
2             Dhahran  Amin bin Hasan Al-Nasser    68,493          NaN  
3  New York, New York               Jamie Dimon   270,000        128.9  
4             Beijing            Jinliang Zhang   347,319          NaN

Также в таблице есть колонки, которые не несут в себе никакой важной информации (ни директор компании, ни местоположения главного офиса ни на что не влияют). Колонки, которые просто держат в курсе, нам не нужны, поэтому удаляем их.

In [ ]:
data = data.drop(columns = ['Headquaters', 'CEO'])

Посмотрим на пропуски

In [ ]:
data.isnull().sum() #удалю строки, в которых не указана Industry, Founded или Employees, т.к. они важны для анализа

rank              0
name              0
country           0
sales             0
profit            0
assets            0
MV                0
Industry        110
Founded           1
Employees         1
Share price    1096
dtype: int64

In [ ]:
data = data.dropna(subset = ['Industry', 'Founded', 'Employees'])

In [ ]:
data['rank'] = data['rank'].astype('int')#перевожу в int просто чтоб красиво было, без десятых долей
data['Founded'] = data['Founded'].astype('int')

В колонке "Share price" очень много пропусков. Заполню их нулями, чтобы можно было как-то анализировать, и добавлю колонку, в которой отражается, торгуются ли акции (1 - торгуются, 0 - не торгуются)

In [ ]:
data['Share price'] = data['Share price'].fillna(0)
data['Shares availability'] = np.where((data['Share price']==0),0,1)

В колонке "Industry" есть некоторые категории,которые дублируются (например, 'E-commerce' и 'E-Commerce). Надо это устранить)

In [ ]:
set(data['Industry']) 

{'Aerospace & Defense',
 'Airlines',
 'Alcoholic Beverages',
 'Appliances',
 'Auto Brands',
 'Auto Repair & Maintenance',
 'Automotive',
 'Automotive (Automotive and Suppliers)',
 'Banking',
 'Banking and Financial Services',
 'Beverages',
 'Business Products & Software Services',
 'Business Services & Supplies',
 'Capital Goods',
 'Casinos & Resorts',
 'Cereals',
 'Chemicals',
 'Clothing, Shoes, Sports Equipment',
 'Conglomerate',
 'Conglomerates',
 'Construction',
 'Construction, Oil & Gas Operations, Mining and Chemicals',
 'Consumer Durables',
 'Credit Card Companies',
 'Cruise Lines',
 'Department Stores',
 'Diversified Financials',
 'Drugs & Biotechnology',
 'Drugs & Biotechnology ',
 'E-Commerce',
 'E-commerce',
 'Electronics',
 'Engineering, Manufacturing',
 'Fashion, Apparel, Shoes',
 'Food & Drink',
 'Food Markets',
 'Food, Drink & Tobacco',
 'Food, Drink and Tobacco',
 'Food, Soft Beverages, Alcohol & Tobacco',
 'Food, Soft Beverages, Alcohol & Tobacco (including agriculture

In [ ]:
data['Industry'] = data['Industry'].apply(lambda x: x.replace('Automotive (Automotive and Suppliers)', 'Automotive'))
data['Industry'] = data['Industry'].apply(lambda x: x.replace('Drugs & Biotechnology ', 'Drugs & Biotechnology'))
data['Industry'] = data['Industry'].apply(lambda x: x.replace('Conglomerates', 'Conglomerate'))
data['Industry'] = data['Industry'].apply(lambda x: x.replace('E-commerce', 'E-Commerce'))
data['Industry'] = data['Industry'].apply(lambda x: x.replace('Food, Drink and Tobacco', 'Food, Drink & Tobacco'))
data['Industry'] = data['Industry'].apply(lambda x: x.replace('Food, Soft Beverages, Alcohol & Tobacco (including agriculture production)', 'Food, Drink & Tobacco'))
data['Industry'] = data['Industry'].apply(lambda x: x.replace('Transportation and Logistics', 'Transportation & Logistics'))
data['Industry'] = data['Industry'].apply(lambda x: x.replace('Travel & Leisure\xa0\xa0\xa0\xa0\xa0\xa0', 'Travel & Leisure'))

Для красоты сделаем все названия колонок с заглавной буквы

In [ ]:
data.rename(columns = {'rank':'Rank', 'name':'Name', 'country':'Country','sales':'Sales', 'profit':'Profit', 'assets':'Assets'}, inplace = True )

In [ ]:
data.head() #готово)

Rank                                      Name        Country     Sales  \
0     1                        Berkshire Hathaway  United States  276090.0   
2     3  Saudi Arabian Oil Company (Saudi Aramco)   Saudi Arabia  400380.0   
3     4                            JPMorgan Chase  United States  124540.0   
5     6                                    Amazon  United States  469820.0   
6     7                                     Apple  United States  378700.0   

     Profit     Assets         MV                Industry  Founded  Employees  \
0   89800.0   958780.0   741480.0  Diversified Financials     1939    372,000   
2  105360.0   576040.0  2292080.0    Oil & Gas Operations     1933     68,493   
3   42120.0  3954690.0   374450.0                 Banking     2000    270,000   
5   33360.0   420550.0  1468400.0              E-Commerce     1994  1,608,000   
6  100560.0   381190.0  2640320.0             Electronics     1976    164,000   

   Share price  Shares availability  
0        318.0                    1  
2          0.0                    0  
3        128.9                    1  
5        102.4                    1  
6        165.5                    1

In [ ]:
data.to_csv('forbes_obrab.csv', sep=',', index=False, lineterminator='')